# NL2SQL Pipeline Demo - Step by Step

## Overview
This notebook demonstrates the **Natural Language to SQL (NL2SQL)** pipeline capabilities.

### What This Pipeline Does:
1. 🧠 **Extract Intent & Entities** from natural language questions
2. 📊 **Read Database Schema** to understand available tables and relationships
3. ⚡ **Generate T-SQL** queries using AI (Azure OpenAI)
4. 🔍 **Sanitize & Validate** the generated SQL
5. 🎯 **Execute** against Azure SQL Database
6. 📈 **Format & Display** results with token usage and cost tracking

### Technologies Used:
- **Azure OpenAI** (GPT-4 or GPT-5/o-series models)
- **LangChain** for AI orchestration
- **Azure SQL Database** for data storage
- **Python 3.11+**

---
## Step 1: Import Dependencies and Load Configuration

First, we'll import all necessary libraries and load our Azure credentials from the `.env` file.

In [1]:
import os
import re
import sys
import time
import json
from typing import List, Dict, Any, Optional
from datetime import datetime
from dotenv import load_dotenv

from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Load environment variables
load_dotenv()

print("✅ Dependencies loaded successfully!")

✅ Dependencies loaded successfully!


---
## Step 2: Configure Azure OpenAI Connection

We'll set up the connection to Azure OpenAI and detect whether we're using a reasoning model (o-series/GPT-5) or a standard model.

In [2]:
# Azure OpenAI configuration
API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2025-04-01-preview")

# Helper: detect reasoning models (o-series and gpt-5)
def _is_reasoning_like_model(deployment_name: str | None) -> bool:
    name = (deployment_name or "").lower()
    return name.startswith("o") or name.startswith("gpt-5")

_USING_REASONING_STYLE = _is_reasoning_like_model(DEPLOYMENT_NAME)

# Display configuration
print("═" * 60)
print("🤖 MODEL CONFIGURATION")
print("═" * 60)
print(f"Deployment: {DEPLOYMENT_NAME}")
print(f"Type: {'Reasoning Model (o-series/GPT-5)' if _USING_REASONING_STYLE else 'Standard Model'}")
print(f"Endpoint: {ENDPOINT}")
print(f"API Version: {API_VERSION}")
print("═" * 60)

════════════════════════════════════════════════════════════
🤖 MODEL CONFIGURATION
════════════════════════════════════════════════════════════
Deployment: gpt-4.1
Type: Standard Model
Endpoint: https://aq-ai-foundry-sweden-central.openai.azure.com/
API Version: 2025-04-01-preview
════════════════════════════════════════════════════════════


---
## Step 3: Initialize the LLM

Create the Language Model instance. For reasoning models, we use direct API calls. For standard models, we use LangChain's wrapper.

In [3]:
def _make_llm():
    """Create an LLM instance based on model type."""
    if _USING_REASONING_STYLE:
        return None  # Use direct Azure Chat Completions calls
    return AzureChatOpenAI(
        openai_api_key=API_KEY,
        azure_endpoint=ENDPOINT,
        deployment_name=DEPLOYMENT_NAME,
        api_version=API_VERSION,
        max_tokens=8192,
    )

llm = _make_llm()

if llm:
    print("✅ LangChain LLM initialized successfully!")
else:
    print("✅ Direct API mode enabled for reasoning model!")

✅ LangChain LLM initialized successfully!


---
## Step 4: Token Usage Tracking

Set up token usage tracking to monitor costs and performance.

In [4]:
# Token usage accumulator
_TOKEN_USAGE = {"prompt": 0, "completion": 0, "total": 0}

def _accumulate_usage(usage: Optional[Dict[str, Any]]) -> None:
    """Accumulate token usage across all API calls."""
    if not usage:
        return
    _TOKEN_USAGE["prompt"] += int(usage.get("prompt_tokens", 0) or 0)
    _TOKEN_USAGE["completion"] += int(usage.get("completion_tokens", 0) or 0)
    _TOKEN_USAGE["total"] += int(usage.get("total_tokens", 0) or (_TOKEN_USAGE["prompt"] + _TOKEN_USAGE["completion"]))

def reset_token_usage():
    """Reset token usage counters."""
    _TOKEN_USAGE["prompt"] = 0
    _TOKEN_USAGE["completion"] = 0
    _TOKEN_USAGE["total"] = 0

print("✅ Token tracking initialized!")

✅ Token tracking initialized!


---
## Step 5: Load Pricing Configuration

Load token pricing from configuration file to calculate costs.

In [5]:
def _load_pricing_config() -> Dict[str, Dict[str, float]]:
    """Load pricing configuration from azure_openai_pricing.json."""
    try:
        pricing_file = os.path.join(os.path.dirname(os.path.abspath('')), "azure_openai_pricing.json")
        if os.path.exists(pricing_file):
            with open(pricing_file, "r") as f:
                return json.load(f)
    except Exception as e:
        print(f"⚠️ Could not load pricing config: {e}")
    return {}

pricing_config = _load_pricing_config()
print(f"✅ Pricing configuration loaded: {len(pricing_config)} deployment(s) configured")

✅ Pricing configuration loaded: 0 deployment(s) configured


---
## Step 6: Define Sample Queries

Let's define some example natural language queries to demonstrate the pipeline.

In [6]:
# Sample queries for demonstration
SAMPLE_QUERIES = [
    "How many customers do I have?",
    "Show me the top 5 loans by outstanding balance",
    "What is the total collateral value across all loans?",
    "List all companies in the Technology sector",
    "Which region has the most loan applications?"
]

print("📋 Sample Queries:")
for i, query in enumerate(SAMPLE_QUERIES, 1):
    print(f"  {i}. {query}")

📋 Sample Queries:
  1. How many customers do I have?
  2. Show me the top 5 loans by outstanding balance
  3. What is the total collateral value across all loans?
  4. List all companies in the Technology sector
  5. Which region has the most loan applications?


---
## Step 7: Intent Extraction Function

This function uses AI to extract the intent and entities from a natural language question.

In [7]:
def extract_intent(query: str) -> str:
    """Extract intent and entities from natural language query."""
    
    intent_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an AI assistant that extracts the intent and entities from natural language database queries.
        
Analyze the user's question and provide:
1. The main intent (e.g., count, list, aggregate, filter)
2. Key entities mentioned (tables, columns, metrics)
3. Any filters or conditions
4. Desired aggregations or groupings

Return your analysis in JSON format with keys: intent, entity, metrics, filters, group_by."""),
        ("user", "{query}")
    ])
    
    if llm:
        chain = intent_prompt | llm
        result = chain.invoke({"query": query})
        return result.content
    else:
        # Direct API call for reasoning models
        import requests
        headers = {
            "Content-Type": "application/json",
            "api-key": API_KEY,
        }
        messages = [
            {"role": "system", "content": intent_prompt.messages[0].prompt.template},
            {"role": "user", "content": query}
        ]
        data = {"messages": messages, "max_tokens": 2000}
        
        url = f"{ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version={API_VERSION}"
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        result = response.json()
        
        # Track usage
        if "usage" in result:
            _accumulate_usage(result["usage"])
        
        return result["choices"][0]["message"]["content"]

print("✅ Intent extraction function ready!")

✅ Intent extraction function ready!


---
## Step 8: Demo - Extract Intent from a Query

Let's see the intent extraction in action!

In [8]:
# Choose a sample query
demo_query = SAMPLE_QUERIES[0]  # "How many customers do I have?"

print("═" * 60)
print("🔍 DEMO: INTENT EXTRACTION")
print("═" * 60)
print(f"\n📝 Query: {demo_query}\n")

# Reset token usage for this demo
reset_token_usage()

# Extract intent
start_time = time.time()
intent_result = extract_intent(demo_query)
elapsed = time.time() - start_time

print("🧠 EXTRACTED INTENT & ENTITIES:")
print("─" * 60)
print(intent_result)
print("─" * 60)
print(f"\n⏱️ Time taken: {elapsed:.2f} seconds")
print(f"📊 Tokens used: {_TOKEN_USAGE['total']} (prompt: {_TOKEN_USAGE['prompt']}, completion: {_TOKEN_USAGE['completion']})")

════════════════════════════════════════════════════════════
🔍 DEMO: INTENT EXTRACTION
════════════════════════════════════════════════════════════

📝 Query: How many customers do I have?

🧠 EXTRACTED INTENT & ENTITIES:
────────────────────────────────────────────────────────────
{
  "intent": "count",
  "entity": ["customers"],
  "metrics": ["count"],
  "filters": [],
  "group_by": []
}
────────────────────────────────────────────────────────────

⏱️ Time taken: 1.29 seconds
📊 Tokens used: 0 (prompt: 0, completion: 0)
🧠 EXTRACTED INTENT & ENTITIES:
────────────────────────────────────────────────────────────
{
  "intent": "count",
  "entity": ["customers"],
  "metrics": ["count"],
  "filters": [],
  "group_by": []
}
────────────────────────────────────────────────────────────

⏱️ Time taken: 1.29 seconds
📊 Tokens used: 0 (prompt: 0, completion: 0)


---
## Step 9: Import Schema Reader

The schema reader helps the AI understand the database structure.

In [11]:
try:
    from schema_reader import get_sql_database_schema_context
    print("✅ Schema reader imported successfully!")
    
    # Get a sample of the schema
    schema_context = get_sql_database_schema_context()
    print(f"\n📊 Schema loaded: {len(schema_context)} characters")
    print(f"\nPreview (first 500 chars):\n{schema_context[:500]}...")
    
except ImportError as e:
    print(f"⚠️ Could not import schema_reader: {e}")
    print("Note: This is expected if running outside the main directory")
except Exception as e:
    print(f"⚠️ Error loading schema: {e}")
    print("The pipeline will continue in generic mode without schema context")
    schema_context = ""

✅ Schema reader imported successfully!

📊 Schema loaded: 4777 characters

Preview (first 500 chars):
DATABASE: CONTOSO-FI (Azure SQL)
GUIDELINES
- Prefer dbo.vw_LoanPortfolio for simple portfolio-style questions.
- For hard/complex questions, use the base tables (Loan, Company, Collateral, Covenant, PaymentSchedule, etc.) and generate SQL with multiple joins, subqueries, CTEs, or advanced logic as needed.
- Use appropriate GROUP BY for aggregates; weight interest rate averages by PrincipalAmount if needed.
- Do not emit USE or GO; return a single executable SELECT statement.

VIEW
- dbo.vw_Loan...


---
## Step 10: SQL Generation Function

This function generates T-SQL based on the intent and database schema.

In [12]:
def generate_sql(intent: str, schema_context: str = "") -> str:
    """Generate SQL query based on extracted intent and schema."""
    
    sql_prompt = ChatPromptTemplate.from_messages([
        ("system", f"""You are an expert SQL query generator for Azure SQL Database.

Given the user's intent and the database schema, generate a valid T-SQL query.

Database Schema:
{schema_context or 'Schema not provided - use generic table names'}

Requirements:
- Generate clean, efficient T-SQL
- Use proper JOINs when needed
- Include appropriate WHERE clauses for filters
- Use meaningful column aliases
- Return ONLY the SQL query, no explanations
"""),
        ("user", "Intent: {intent}\n\nGenerate the SQL query:")
    ])
    
    if llm:
        chain = sql_prompt | llm
        result = chain.invoke({"intent": intent})
        return result.content
    else:
        # Direct API call for reasoning models
        import requests
        headers = {
            "Content-Type": "application/json",
            "api-key": API_KEY,
        }
        messages = [
            {"role": "system", "content": sql_prompt.messages[0].prompt.template.format(schema_context=schema_context or 'Schema not provided')},
            {"role": "user", "content": f"Intent: {intent}\n\nGenerate the SQL query:"}
        ]
        data = {"messages": messages, "max_tokens": 2000}
        
        url = f"{ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version={API_VERSION}"
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        result = response.json()
        
        # Track usage
        if "usage" in result:
            _accumulate_usage(result["usage"])
        
        return result["choices"][0]["message"]["content"]

print("✅ SQL generation function ready!")

✅ SQL generation function ready!


---
## Step 11: SQL Sanitization

Extract clean SQL from the AI response (removes markdown formatting, etc.)

In [13]:
def extract_and_sanitize_sql(raw_sql: str) -> str:
    """Extract and clean SQL from AI response."""
    # Remove markdown code blocks
    sql = re.sub(r'^```(?:sql)?\s*', '', raw_sql, flags=re.MULTILINE)
    sql = re.sub(r'```\s*$', '', sql, flags=re.MULTILINE)
    
    # Trim whitespace
    sql = sql.strip()
    
    return sql

print("✅ SQL sanitization function ready!")

✅ SQL sanitization function ready!


---
## Step 12: Demo - Full Pipeline

Now let's run the complete pipeline from natural language to SQL!

In [17]:
def run_nl2sql_pipeline(query: str, execute: bool = False):
    """Run the complete NL2SQL pipeline."""
    
    print("═" * 70)
    print("🚀 COMPLETE NL2SQL PIPELINE")
    print("═" * 70)
    
    # Reset token usage
    reset_token_usage()
    start_time = time.time()
    
    # Step 1: Display query
    print(f"\n📝 NATURAL LANGUAGE QUERY")
    print("─" * 70)
    print(query)
    
    # Step 2: Extract intent
    print(f"\n\n🧠 STEP 1: EXTRACT INTENT & ENTITIES")
    print("─" * 70)
    intent = extract_intent(query)
    print(intent)
    
    # Step 3: Get schema (if available)
    print(f"\n\n📊 STEP 2: LOAD DATABASE SCHEMA")
    print("─" * 70)
    try:
        schema = get_sql_database_schema_context()
        print(f"✅ Schema loaded ({len(schema)} characters)")
    except Exception as e:
        schema = ""
        print(f"⚠️ Schema not available (using generic mode): {e}")
    
    # Step 4: Generate SQL
    print(f"\n\n⚡ STEP 3: GENERATE SQL QUERY")
    print("─" * 70)
    raw_sql = generate_sql(intent, schema)
    print(raw_sql)
    
    # Step 5: Sanitize SQL
    print(f"\n\n🔍 STEP 4: SANITIZE SQL")
    print("─" * 70)
    clean_sql = extract_and_sanitize_sql(raw_sql)
    print(clean_sql)
    
    # Step 6: Execute (if requested)
    if execute:
        print(f"\n\n🎯 STEP 5: EXECUTE QUERY")
        print("─" * 70)
        try:
            from sql_executor import execute_sql_query
            results = execute_sql_query(clean_sql)
            print(f"✅ Query executed successfully! Returned {len(results)} row(s)")
            if results:
                print("\nResults:")
                for i, row in enumerate(results[:5], 1):  # Show first 5 rows
                    print(f"  {i}. {row}")
                if len(results) > 5:
                    print(f"  ... and {len(results) - 5} more row(s)")
        except Exception as e:
            print(f"❌ Execution error: {e}")
    else:
        print(f"\n\n⏭️ STEP 5: EXECUTION SKIPPED")
        print("─" * 70)
        print("Set execute=True to run the query against the database")
    
    # Summary
    elapsed = time.time() - start_time
    print(f"\n\n📈 PIPELINE SUMMARY")
    print("═" * 70)
    print(f"⏱️ Total time: {elapsed:.2f} seconds")
    print(f"📊 Total tokens: {_TOKEN_USAGE['total']} (prompt: {_TOKEN_USAGE['prompt']}, completion: {_TOKEN_USAGE['completion']})")
    
    # Calculate cost if pricing available
    if pricing_config and DEPLOYMENT_NAME:
        dep_key = DEPLOYMENT_NAME.lower()
        if dep_key in pricing_config:
            prices = pricing_config[dep_key]
            input_cost = (_TOKEN_USAGE['prompt'] / 1000.0) * prices.get('input_per_1k', 0)
            output_cost = (_TOKEN_USAGE['completion'] / 1000.0) * prices.get('output_per_1k', 0)
            total_cost = input_cost + output_cost
            print(f"💰 Estimated cost: ${total_cost:.6f} USD")
    
    print("═" * 70)

print("✅ Pipeline function ready!")

✅ Pipeline function ready!


---
## Step 13: Run a Demo Query

Let's test the pipeline with our sample query!

In [18]:
# Run the pipeline with the first sample query
run_nl2sql_pipeline(SAMPLE_QUERIES[0], execute=True)

══════════════════════════════════════════════════════════════════════
🚀 COMPLETE NL2SQL PIPELINE
══════════════════════════════════════════════════════════════════════

📝 NATURAL LANGUAGE QUERY
──────────────────────────────────────────────────────────────────────
How many customers do I have?


🧠 STEP 1: EXTRACT INTENT & ENTITIES
──────────────────────────────────────────────────────────────────────
{
  "intent": "count",
  "entity": "customers",
  "metrics": ["customer_count"],
  "filters": [],
  "group_by": []
}


📊 STEP 2: LOAD DATABASE SCHEMA
──────────────────────────────────────────────────────────────────────
✅ Schema loaded (4777 characters)


⚡ STEP 3: GENERATE SQL QUERY
──────────────────────────────────────────────────────────────────────
{
  "intent": "count",
  "entity": "customers",
  "metrics": ["customer_count"],
  "filters": [],
  "group_by": []
}


📊 STEP 2: LOAD DATABASE SCHEMA
──────────────────────────────────────────────────────────────────────
✅ Schema loaded (

---
## Step 14: Try Your Own Query

Now you can test with your own natural language question!

In [19]:
# Uncomment and modify to try your own query:
# custom_query = "What are the total loan amounts by region?"
# run_nl2sql_pipeline(custom_query, execute=False)

# Or try another sample:
run_nl2sql_pipeline(SAMPLE_QUERIES[1], execute=True)

══════════════════════════════════════════════════════════════════════
🚀 COMPLETE NL2SQL PIPELINE
══════════════════════════════════════════════════════════════════════

📝 NATURAL LANGUAGE QUERY
──────────────────────────────────────────────────────────────────────
Show me the top 5 loans by outstanding balance


🧠 STEP 1: EXTRACT INTENT & ENTITIES
──────────────────────────────────────────────────────────────────────
{
  "intent": "list",
  "entity": ["loans", "outstanding balance"],
  "metrics": ["outstanding balance"],
  "filters": [],
  "group_by": [],
  "sort": {
    "by": "outstanding balance",
    "order": "desc",
    "limit": 5
  }
}


📊 STEP 2: LOAD DATABASE SCHEMA
──────────────────────────────────────────────────────────────────────
✅ Schema loaded (4777 characters)


⚡ STEP 3: GENERATE SQL QUERY
──────────────────────────────────────────────────────────────────────
{
  "intent": "list",
  "entity": ["loans", "outstanding balance"],
  "metrics": ["outstanding balance"],
  "f

---
## 🎉 Demo Complete!

### What We Demonstrated:
1. ✅ Loading and configuring Azure OpenAI
2. ✅ Extracting intent from natural language
3. ✅ Loading database schema context
4. ✅ Generating T-SQL queries using AI
5. ✅ Sanitizing and validating SQL
6. ✅ Tracking token usage and costs

### Key Benefits:
- 🚀 **Speed**: Generate complex queries in seconds
- 💡 **Accessibility**: No SQL knowledge required
- 🎯 **Accuracy**: Schema-aware generation
- 💰 **Cost Tracking**: Monitor API usage and costs
- 🔒 **Safety**: SQL sanitization and validation

### Next Steps:
- Run the complete pipeline with `nl2sql_main.py`
- Integrate with your own applications
- Customize prompts for your specific domain
- Add more sophisticated error handling
- Implement query result caching